# find session with visual area 

In [5]:
from firing_rate import firingRates_onDepths_passiveActive
import numpy as np
from one.api import ONE
import json

# # Initialize ONE with the IBL public server
# one = ONE(base_url='https://openalyx.internationalbrainlab.org')

# # Retrieve all probe insertions that include any of the specified visual areas
# visual_areas = ['VISp', 'VISpm', 'VISam', 'VISa', 'VISrl', 'VISal', 'VISli', 'VISl']
# insertions = []
# for area in visual_areas:
#     insertions += one.alyx.rest('insertions', 'list', task_protocol='ephys', performance_gte=70, 
#                                 dataset_qc_gte='PASS', atlas_acronym=area)

# # Remove duplicates if any
# insertions = {insertion['id']: insertion for insertion in insertions}.values()

# pid_eid_pairs = [(insertion['id'], insertion['session']) for insertion in insertions]
# print(f"Total number of probe insertions: {len(pid_eid_pairs)}")

# eids = [eid for _, eid in pid_eid_pairs]
# no_passive_eids = []
# for eid in eids:
#     datasets = one.list_datasets(eid)
#     datasets_passive = [dataset for dataset in datasets if 'passiveGabor' in dataset]
#     if len(datasets_passive) == 0:
#         no_passive_eids.append(eid)
# print(f"Number of sessions without passive data: {len(no_passive_eids)}")

# eid_with_passive = [eid for eid in eids if eid not in no_passive_eids]
# pid_eid_pairs = [(pid, eid) for pid, eid in pid_eid_pairs if eid in eid_with_passive]
# print(f"Number of sessions with passive data: {len(pid_eid_pairs)}")

# # save the pairs to a json file
# with open('pid_eid_pairs.json', 'w') as f:
#     json.dump(pid_eid_pairs, f)

# load the pairs from the json file
with open('pid_eid_pairs.json', 'r') as f:
    pid_eid_pairs = json.load(f)
print(f"Number of sessions with passive data: {len(pid_eid_pairs)}")

Number of sessions with passive data: 129


In [6]:
t_bin=0.025 # time bins, I  tried 0.02 and wierdly the number of time points was different for active and passive data 
pre_stim=0.8 # time before stimulus onset
post_stim= 0.4 # time after stimulus onset
modee = 'download' # download the spikes
min_contrast= 0 # include all trials with contrasts
probability_left= 'all'
min_time = -0.5
base_path = '/mnt/data/AdaptiveControl/IBLrawdata/classification/preprocess_firingRate'
overwrite = True

from functools import partial
# Create a partial function with fixed parameters
firingRates_partial = partial(firingRates_onDepths_passiveActive, t_bin=t_bin,
                              pre_stim=pre_stim,
                              post_stim=post_stim,
                              modee=modee,
                              min_contrast=min_contrast,
                              probability_left=probability_left,
                              min_time=min_time,
                              base_path=base_path,
                              overwrite=overwrite)


start = 0
end = -1
pid_eid_pairs_short = pid_eid_pairs[start:end]

import submitit
import time
# prepare executor
executor = submitit.AutoExecutor(folder="tuto_logs")
# define maxjobs to a low value to illustrate
maxjobs= 80
# pass parameter to the executor
executor.update_parameters(slurm_array_parallelism=maxjobs, mem_gb=20, timeout_min=300, slurm_partition="CPU", cpus_per_task=1)
# execute the job (note the .map_array command that different from the .submit command used above)
jobs = executor.map_array(firingRates_partial, pid_eid_pairs_short)




In [7]:
# Monitor job status
total_jobs = len(jobs)
print(f"Submitted {total_jobs} jobs.")

finished_jobs = 0
completed_jobs_set = set()

while finished_jobs < total_jobs:
    for idx, job in enumerate(jobs):
        if job.done() and idx not in completed_jobs_set:
            finished_jobs += 1
            completed_jobs_set.add(idx)
            print(f"Jobs finished: {finished_jobs}/{total_jobs}")

print("All jobs are finished.")

Submitted 128 jobs.
Jobs finished: 1/128
Jobs finished: 2/128
Jobs finished: 3/128
Jobs finished: 4/128
Jobs finished: 5/128
Jobs finished: 6/128
Jobs finished: 7/128
Jobs finished: 8/128
Jobs finished: 9/128
Jobs finished: 10/128
Jobs finished: 11/128
Jobs finished: 12/128
Jobs finished: 13/128
Jobs finished: 14/128
Jobs finished: 15/128
Jobs finished: 16/128
Jobs finished: 17/128
Jobs finished: 18/128
Jobs finished: 19/128
Jobs finished: 20/128
Jobs finished: 21/128
Jobs finished: 22/128
Jobs finished: 23/128
Jobs finished: 24/128
Jobs finished: 25/128
Jobs finished: 26/128
Jobs finished: 27/128
Jobs finished: 28/128
Jobs finished: 29/128
Jobs finished: 30/128
Jobs finished: 31/128
Jobs finished: 32/128
Jobs finished: 33/128
Jobs finished: 34/128
Jobs finished: 35/128
Jobs finished: 36/128
Jobs finished: 37/128
Jobs finished: 38/128
Jobs finished: 39/128
Jobs finished: 40/128
Jobs finished: 41/128
Jobs finished: 42/128
Jobs finished: 43/128
Jobs finished: 44/128
Jobs finished: 45/128

In [8]:
alreadydon_jobs = []
mismatch_jobs = []
badturnsgood_jobs = []
unsuccess_jobs = []
for job in jobs:
    try:
        result = job.result()
        if result == 1:
            alreadydon_jobs.append(job)
        elif result == 2:
            mismatch_jobs.append(job)
        elif result == 0:
            badturnsgood_jobs.append(job)
    except:
        unsuccess_jobs.append(job)
        continue
print(f'Already done jobs: {len(alreadydon_jobs)}')
print(f'Mismatch jobs: {len(mismatch_jobs)}')
print(f'Bad turns good jobs: {len(badturnsgood_jobs)}')
print(f'Unsuccess jobs: {len(unsuccess_jobs)}')


Already done jobs: 0
Mismatch jobs: 0
Bad turns good jobs: 127
Unsuccess jobs: 1
